In [1]:
import requests
from lxml import etree
from urllib import request
import os
import re
import threading

## 单线程下载

In [41]:
def parse_page(url):
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    text = response.text
    html = etree.HTML(text)
    imgs = html.xpath("//div[@class='page-content text-center']//img[@class!='gif']")
    for img in imgs:
        alt = img.get('alt')
        alt = re.sub(r'[\?？\.\*，。！!/]', '', alt)
        img_url = img.get('data-original')
        # extension
        suffix = os.path.splitext(img_url)[1]
        filename = alt + suffix
        request.urlretrieve(img_url, 'images/' + filename)

def main():
    for x in range(1, 101):
        url = 'https://www.doutula.com/photo/list/?page=%d' % x
        parse_page(url)

In [42]:
main()

今晚来不来陪陪人家嘛.jpg
取汝底裤犹如探囊取物.jpg
短屌男.jpg
没牌面的小老弟往后捎一捎.jpg
凶猪下下山.jpg
立回无脑下中脚踢中取消连到老.jpg
我要抱抱.jpg
你不要太嚣张.jpg
牙灭蝶.jpg
谁能禁言我我就娶了她.jpg
虾扯蛋.jpg
你想的美.jpg
七减一.jpg
搞事搞事搞事.jpg
你这里是不是有病啊.jpg
是吗我好怕.jpg
喂有人吗开门啊.jpg
我看你是土狗打饱嗝屎吃多了.jpg
真受不了你这人.gif
平胸老阿姨.jpg
不能停多吃点.jpg
这是我女仆.jpg
呀咩咯欧捏该.jpg
你说的话我连标点符号都不信.jpg
亚咩漏.jpg
你很棒棒哦.jpg
五百年了.jpg
蹦蹦蹦.jpg
咕叽咕叽.jpg
随风奔跑自由是方向追逐雷和闪电的力量.gif
养你的猪去吧.jpg
我这该死的帅哥.jpg
OK呀.jpg
理直气壮1500工资找女朋友.jpg
我能插嘴吗.jpg
你能不能像我这样成熟点.jpg
就你皮.jpg
能借七块给我吗我想买瓶红牛.jpg
瞧你那熊样.jpg
禁我啊来啊.jpg
佐住你那屁股眼.jpg
发呆.jpg
男人想肾好就要活肾宝.jpg
死会儿.jpg
滚开有鸡巴的女人.jpg
你妈.jpg
来叫声主.jpg
气哭.jpg
今晚让你爽翻天.jpg
打开本群空调.jpg
你真是一个奶子又小打游戏又菜的女孩子.jpg
没毛病.jpg
我操.jpg
他骗你.jpg
卧槽我一下慌了.jpg
我要自杀让你没有爸爸.jpg
这个就有点.jpg
每当群里没人的时候老子帅气登场.jpg
美人鱼有没有.jpg
水蛇腰有没有.jpg
我喝多了（小猫）.jpg
呵呵小场面.jpg
国服马可.jpg
国服武则天.jpg
智力-1000.jpg
不要动.jpg
大佬.jpg
信不信把你的逼儿舔得嗷嗷叫.jpg
你是个大便子.gif
再这样结束你处女生活.jpg
喂出来赌博啊.jpg
你说什么我听不懂.gif
真恶心.jpg
大不服的都可以站出来不论男女老少.jpg
从今天开始我就是这个群的老大不服的都站出来.jpg
打倒狗庄.jpg
不看书了去玩女管理.jpg
我的图我的图.jpg
吓得我都发癫了.jpg
吧唧吧唧.jpg
一位不愿露面的网友前来捧场.jpg
抱歉不能.jpg
没错就是你.jpg
刺溜.jpg
听听你说的

KeyboardInterrupt: 

## 多线程下载
`每一页的url -> 生产者（获取表情url）-> 每个表情的url -> 消费者（下载表情）`

In [2]:
import requests
from lxml import etree
from urllib import request
import os
import re
from queue import Queue
import threading

In [9]:
class Producer(threading.Thread):
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    }
    def __init__(self, page_queue, img_queue ,*args, **kwargs):
        super(Producer, self).__init__(*args, **kwargs)
        self.page_queue = page_queue
        self.img_queue = img_queue
    
    def run(self):
        while True:
            if self.page_queue.empty():
                break
            url = self.page_queue.get()
            self.parse_page(url)

    def parse_page(self, url):
        response = requests.get(url, headers=self.headers)
        text = response.text
        html = etree.HTML(text)
        imgs = html.xpath("//div[@class='page-content text-center']//img[@class!='gif']")
        for img in imgs:
            alt = img.get('alt')
            alt = re.sub(r'[\?？\.\*，。！!/]', '', alt)
            img_url = img.get('data-original')
            # extension
            suffix = os.path.splitext(img_url)[1]
            filename = alt + suffix
            self.img_queue.put((img_url, filename))

class Consumer(threading.Thread):
    def __init__(self, page_queue, img_queue ,*args, **kwargs):
        super(Consumer, self).__init__(*args, **kwargs)
        self.page_queue = page_queue
        self.img_queue = img_queue
        
    def run(self):
        while True:
            if self.img_queue.empty() and self.page_queue.empty():
                break
            img_url, filename = self.img_queue.get()
            request.urlretrieve(img_url, 'images/' + filename)
            print(filename + '下载完成!')
            
            
def main():
    page_queue = Queue(100)
    img_queue = Queue(1000)
    for x in range(1, 101):
        url = 'https://www.doutula.com/photo/list/?page=%d' % x
        page_queue.put(url)
    
    for x in range(5):
        t = Producer(page_queue, img_queue)
        t.start()
    
    for x in range(5):
        t = Consumer(page_queue, img_queue)
        t.start()

In [10]:
main()

没牌面的小老弟往后捎一捎.jpg下载完成!
短屌男.jpg下载完成!
凶猪下下山.jpg下载完成!取汝底裤犹如探囊取物.jpg下载完成!

今晚来不来陪陪人家嘛.jpg下载完成!
立回无脑下中脚踢中取消连到老.jpg下载完成!
我要抱抱.jpg下载完成!
牙灭蝶.jpg下载完成!
谁能禁言我我就娶了她.jpg下载完成!
你不要太嚣张.jpg下载完成!
虾扯蛋.jpg下载完成!
你想的美.jpg下载完成!
七减一.jpg下载完成!
搞事搞事搞事.jpg下载完成!
你这里是不是有病啊.jpg下载完成!
是吗我好怕.jpg下载完成!
喂有人吗开门啊.jpg下载完成!
我看你是土狗打饱嗝屎吃多了.jpg下载完成!
真受不了你这人.gif下载完成!
这是我女仆.jpg下载完成!
平胸老阿姨.jpg下载完成!不能停多吃点.jpg下载完成!

呀咩咯欧捏该.jpg下载完成!
你说的话我连标点符号都不信.jpg下载完成!
亚咩漏.jpg下载完成!
你很棒棒哦.jpg下载完成!五百年了.jpg下载完成!

蹦蹦蹦.jpg下载完成!咕叽咕叽.jpg下载完成!

我这该死的帅哥.jpg下载完成!
随风奔跑自由是方向追逐雷和闪电的力量.gif下载完成!
养你的猪去吧.jpg下载完成!
OK呀.jpg下载完成!理直气壮1500工资找女朋友.jpg下载完成!

就你皮.jpg下载完成!
能借七块给我吗我想买瓶红牛.jpg下载完成!
我能插嘴吗.jpg下载完成!你能不能像我这样成熟点.jpg下载完成!瞧你那熊样.jpg下载完成!


禁我啊来啊.jpg下载完成!
发呆.jpg下载完成!
佐住你那屁股眼.jpg下载完成!
死会儿.jpg下载完成!
男人想肾好就要活肾宝.jpg下载完成!
滚开有鸡巴的女人.jpg下载完成!
你妈.jpg下载完成!
来叫声主.jpg下载完成!
打开本群空调.jpg下载完成!
今晚让你爽翻天.jpg下载完成!
气哭.jpg下载完成!
你真是一个奶子又小打游戏又菜的女孩子.jpg下载完成!
没毛病.jpg下载完成!
他骗你.jpg下载完成!
我操.jpg下载完成!
卧槽我一下慌了.jpg下载完成!
我要自杀让你没有爸爸.jpg下载完成!
这个就有点.jpg下载完成!
每当群里没人的时候老子帅气登场.jpg下载完成!
美人鱼有没有.jpg下载完成!
水蛇腰有没有.jpg下载

Exception in thread Thread-29:
Traceback (most recent call last):
  File "/Users/Daiii/anaconda3/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Daiii/anaconda3/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Daiii/anaconda3/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Daiii/anaconda3/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Daiii/anaconda3/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Daiii/anaconda3/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Daiii/anaconda3/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/Users/Daiii/anaco

害怕疑问.jpg下载完成!
打头.jpg下载完成!
日自己叫日本人.jpg下载完成!
花Q.jpg下载完成!
啊北仔.jpg下载完成!
我是你爹.jpg下载完成!
我是gay.jpg下载完成!
坏坏打拍子.gif下载完成!
狗东西.jpg下载完成!
咬人.jpg下载完成!
混账这就是你跟爹说话的口气.jpg下载完成!
管理呢给我买包烟去.jpg下载完成!
你当你是谁啊.jpg下载完成!死靓仔看到没女管理爱的是我.jpg下载完成!

撤回也没用我看到了.jpg下载完成!老铁借点钱梭哈赢了双倍返还输了就当交个朋友好了.jpg下载完成!

记住我脸上的伤疤和我的故事.jpg下载完成!
隔壁的老王.jpg下载完成!
告辞.jpg下载完成!
向基佬势力低头.jpg下载完成!
渣渣面对疾风吧.jpg下载完成!
刷卡买买买.gif下载完成!
摇头.gif下载完成!
闻屎.gif下载完成!
头掉了.gif下载完成!
抱着你不哭.gif下载完成!
跳舞可爱妹子.gif下载完成!
配钥匙.jpg下载完成!
doge念经.gif下载完成!
小本经营（杂粮煎饼）.gif下载完成!
注意你的言辞.jpg下载完成!
开学啦.jpg下载完成!
恰恰相反.jpg下载完成!
叮咚叮咚.jpg下载完成!
令人费解.jpg下载完成!
方便的话能请你去死一下吗.jpg下载完成!
李遐怡我爱你.jpg下载完成!女权主义、中华田园女权.jpg下载完成!

怕了怕.jpg下载完成!
笑容再次绽放.jpg下载完成!
噗（爱心）.jpg下载完成!
摸一下小可爱.jpg下载完成!这破鸡巴群迟早要完.jpg下载完成!

管理呢给我买包烟去.jpg下载完成!
各部门请注意今晚全体加班到凌晨.jpg下载完成!好笑吗傻逼.jpg下载完成!

总感觉你们在背着我学习.jpg下载完成!
少年玩心么.jpg下载完成!
哈哈哈哈呢.jpg下载完成!
丧尽天良.jpg下载完成!
你可以睡我但不可以睡我的这些枕头.jpg下载完成!
你踏马有我屌.jpg下载完成!
要我一直笑吗.jpg下载完成!
失败早睡计划启动.jpg下载完成!
我是群里存在感最低的.jpg下载完成!
你好香啊.jpg下载完成!
人呢.jpg下载完成!
好饿啊早知道不充王者了.jpg下载完成!
几把那么小就别调皮了好吗.jpg下载完成!
咬牙切齿你真是一个又皮又欠收抬的

再叫拿枪打你鸡巴.jpg下载完成!
就问你一句服不服.jpg下载完成!
苦也一天笑也一天快乐就好.jpg下载完成!
装逼王装逼没人是我的对手.jpg下载完成!
玉兰油（面膜）.jpg下载完成!
还不是因为你丑.jpg下载完成!
被这么多人骂还不赶快退群脸皮比包皮还厚.jpg下载完成!
雪花秀（面膜）.jpg下载完成!
你麻痹不是帅哥就不要艾特我.jpg下载完成!
温碧泉（面膜）.jpg下载完成!
摊手.jpg下载完成!
扎心了老铁.jpg下载完成!
聊天结束.jpg下载完成!
以后看到这个人直接打（省长）.jpg下载完成!
oh fuck.jpg下载完成!
你觉得呢.jpg下载完成!
你骂我我不听你妈是个白骨精白骨精黑爪子扯你爸的裤衩子.jpg下载完成!
我在.jpg下载完成!
以后看到这个人直接打（管理员）.jpg下载完成!
遭被dalao发现了.jpg下载完成!
乡下来的多多关照.jpg下载完成!
菜到去世.jpg下载完成!
想battle吗.gif下载完成!能做到這樣才能被稱為〃男生〃.jpg下载完成!

我就静静地看着你装逼.jpg下载完成!
做我的小骚逼吧我给你冲ＱＱ会员.jpg下载完成!
我吃柠檬告辞.jpg下载完成!
可笑剑魂.jpg下载完成!
you bad bad你坏坏.jpg下载完成!
进群1天.jpg下载完成!
今晚吃鸡大吉大利吃鸡不可怕没谁谁尴尬.jpg下载完成!
今天的发票开了吗.jpg下载完成!
像这种傻逼活在世上还有救么.jpg下载完成!
我特么掐死你.jpg下载完成!
你他妈来不来.jpg下载完成!
马上给我道歉.jpg下载完成!
不是很懂你们狗管理.jpg下载完成!
SK2.jpg下载完成!
偷抹眼泪.jpg下载完成!
你居然吼我.jpg下载完成!
我要拉黑你你这个坏女人.jpg下载完成!
都老夫老妻了计较什么.jpg下载完成!
嗤之以鼻.jpg下载完成!
文明人聊天归聊天注意素质好吧.jpg下载完成!
拿烟的手微微ＯＫ.jpg下载完成!去尼玛的.jpg下载完成!

吃屎啦你.jpg下载完成!
人在广东已经嫖到失联…….jpg下载完成!
你什么时候来我.jpg下载完成!暖群神器一分钟之内没人说话我就是你们的爸爸.jpg下载完成!
此话无效.jpg下载完成!

你要不要小黄书.jpg下载完成!
真的分手了吗我错了我道歉.jpg下载完成

超巨大坏坏.jpg下载完成!
我踏马上去给你一斧子.jpg下载完成!
严肃点.jpg下载完成!
low死了.jpg下载完成!
再说一遍做我的走狗这两包都是你的.jpg下载完成!
你不行啊.gif下载完成!
熊猫人委屈.jpg下载完成!被绿了吧.jpg下载完成!

歪我在Ｐ城被人包了你们什么时候来接我啊.jpg下载完成!
大神大神.jpg下载完成!
嚣张.jpg下载完成!
给大佬上香.jpg下载完成!
真的不骗你.jpg下载完成!
好好学习努力发图.jpg下载完成!
拆车.gif下载完成!
不管你是谁只要你送我这个我们就是永远的朋友（RTX 2080Ti）.jpg下载完成!
肥宅.jpg下载完成!
穷带有吗.jpg下载完成!
人生啊.jpg下载完成!
斗图天尊我呸.jpg下载完成!
你算哪片姜.jpg下载完成!
你的淫荡超乎我想象.jpg下载完成!
不简单.jpg下载完成!
这么说你是不打算给小可爱面子咯.jpg下载完成!
我今天不挫挫你棱角你就不知道什么叫社会.jpg下载完成!
结什么婚没听过婚姻是爱情的坟墓.jpg下载完成!
吃鸡吗.jpg下载完成!
拜托你们都很弱唉.gif下载完成!
尴尬了.jpg下载完成!
几个丑东西看着就烦.jpg下载完成!
你个猪脑壳.jpg下载完成!
问题是没钱.jpg下载完成!
做人最要紧的就是开心.jpg下载完成!
gay的一批gay的一批.jpg下载完成!
不昵崛相的小埋.jpg下载完成!
我长得可爱不说也比你们的大简直就是女神.jpg下载完成!
回猪圈去.jpg下载完成!
跪下了.gif下载完成!
堆稽如山（滑稽）.jpg下载完成!
你他妈倒是车啊.jpg下载完成!
哮天犬；我们走.gif下载完成!
不知道用什么表情面对开学.jpg下载完成!
当时我就是这样锁住狗管理的喉.jpg下载完成!
背锅.jpg下载完成!
此时一位靓仔炫酷出现.jpg下载完成!
惊讶.jpg下载完成!
早大家早.jpg下载完成!
七夕节快乐快乐.jpg下载完成!
美少女从不说谎.jpg下载完成!
吹风扇.gif下载完成!
除了白眼宝宝什么都不会给你的.jpg下载完成!
穿内裤.jpg下载完成!
我搂着你.jpg下载完成!
就没人理他一下吗他现在好尴尬.jpg下载完成!
大吉吧.jpg下载完成!
撑紫色伞.jpg下载完成!
吃东西.jpg下载完成!
这这是

二狗还在这装逼呢老板叫你快去搬砖.gif下载完成!
二货.gif下载完成!
学习使我感到快乐参.jpg下载完成!
妈蛋掌嘴.jpg下载完成!
此时一位靓仔踏着群主路过.gif下载完成!
当然是母爱.gif下载完成!
开车开车.jpg下载完成!
丑只是一方面你还蠢.gif下载完成!
狗咬狗精彩的.gif下载完成!
等我跟别人在一起了你会后悔的.gif下载完成!
扑街吓到我飚冷汗.gif下载完成!
发动魔法卡「食屎啦你」.gif下载完成!
呜呜呜甲应.gif下载完成!
叫你们这儿的头牌出来接客.gif下载完成!
吃鸡吃鸡.gif下载完成!
出轨找我我有钱名牌包包任你选.gif下载完成!
你看有人理你吗.gif下载完成!
睡鸡毛凌晨５点还不到就睡睡鸟啊.jpg下载完成!
你是不是傻逼你自己想想.jpg下载完成!
拖出去阉了.jpg下载完成!
好犀利啊.gif下载完成!
网恋选我我超甜想买东西没有钱~.gif下载完成!
一起辞职.gif下载完成!
给我一夜时闻换你十月安稳.gif下载完成!
可以借你的手帮我撸管吗.gif下载完成!
你们这些暴民我每天来就是99+安静点不行啊.jpg下载完成!
我们这里昼夜温差大我超甜的.gif下载完成!
忘了他吧我带你上黄金.jpg下载完成!
卡哇伊呢.jpg下载完成!
妖魔鬼怪快离开.gif下载完成!
我怎么感觉我不是你宝贝了.jpg下载完成!
操你一顿你就舒服了.jpg下载完成!
收声.jpg下载完成!
怪莫怪.gif下载完成!
臭狗.jpg下载完成!
看看我爱的人在不在不在的话我晚点来.gif下载完成!
你特么男的女的.jpg下载完成!
修仙中.gif下载完成!
别惹我不然你就常回家看看.gif下载完成!
可达鸭懵逼.jpg下载完成!
你再说我就发自拍了.jpg下载完成!
拜托你饶了我吧.gif下载完成!
我不是你的小甜甜了吗.jpg下载完成!
持受社会毒打.jpg下载完成!
我不想打击你劝你也适可而止.gif下载完成!
牙一扒子外死你.jpg下载完成!
你们都退群吧让我一个人安静下.gif下载完成!
你是没挨过揍吗.jpg下载完成!
总有儿子骂爸爸.jpg下载完成!
你把我召唤出来了你要实现我１００个愿望.jpg下载完成!
咋肥四捏.jpg下载完成!
泰拳警告.gif下载完成!
希皮笑脸.jpg下载完成!
你觉得我会有空吗.gif下